# Fluoro-changing Tags Based on DNA Strand Displacement Enable Highly Multiplexed Protein Imaging
## Overview of FcTags
Fluoro-changing Tags (FcTags) are capable of generating exponentially scalable antibody tags for spatial proteomics imaging with easily accessible equipment and reagent. To decode proteins from FcTag-based images, we developed an AI algorithm platform here. The algorithm accurately aligns fluorescent images captured at different time points, and decodes proteins via two unique algorithms called `Subtraction` and `Linear unmixing`. The step-by-step operations and important notes can be found in the corresponding section of each algorithm file.

## Computer hardware (optional)
- Computer workstation equipped with an AMD Ryzen 5975WX CPU
- NVIDIA RTX 3090 graphics processing card

## Environmental setup
See pyproject.toml file for library version and various dependencies. Here are the steps you need to take to install the python environment:

### 1. Install Jupyter Notebook 7.x (or Python 3.x)
The algorithm is crafted in Python and executed within Jupyter Notebook.  To execute the files, you have the option to install either `Python 3.x` or `Jupyter Notebook 7.x`.

To install Jupyter Notebook 7.x, you can download it from the [official Jupyter website](https://jupyter.org/).

To install Python 3.x, you can download it from the [official Python website](https://www.python.org/).

### 2. Update `pip` (optional)
It's a good practice to keep `pip` up to date. Open a terminal (or command prompt) and run the following command:

```bash
python -m pip install --upgrade pip
```

### 3. Install Required Libraries
To install the necessary packages (`numpy`, `PIL`, `cv2`, `pandas`), run the following command in your terminal or command prompt:

```bash
pip install numpy scipy scikit-image pandas
```

### 4. Verify Installation
After installation, you can verify that the libraries are installed correctly by running this Python script:

```python
import numpy as np
import cv2
import pandas as pd

print(f"numpy version: {np.__version__}")
print(f"cv2 version: {cv2.__version__}")
print(f"pandas version: {pd.__version__}")
```

This script will print the installed versions of the libraries. If you see the version numbers printed without any errors, the libraries were installed successfully.

## Step 1: Preparing input images
Image obtained at time point 1 (T1)
- `T1_DAPI`: DAPI channel obtained at T1 (0 min in this study);
- `T1_AF488`: AF488 channel obtained at T1 (0 min in this study);
- `T1_Cy3`: Cy3 channel obtained at T1 (0 min in this study);
- `T1_Cy5`: Cy5 channel obtained at T1 (0 min in this study);<br>

Image obtained at time point 2 (T2)
- `T2_DAPI`: DAPI channel obtained at T2 (15 min in this study);
- `T2_AF488`: AF488 channel obtained at T2 (15 min in this study);
- `T2_Cy3`: Cy3 channel obtained at T2 (15 min in this study);
- `T2_Cy5`: Cy5 channel obtained at T2 (15 min in this study);<br>

Image obtained at time point 3 (T3)
- `T3_DAPI`: DAPI channel obtained at T3 (30 min in this study);
- `T3_AF488`: AF488 channel obtained at T3 (30 min in this study);
- `T3_Cy3`: Cy3 channel obtained at T3 (30 min in this study);
- `T3_Cy5`: Cy5 channel obtained at T3 (30 min in this study);<br>

Notes：
- All images are captured under identical microscopic parameters.
- The DAPI channel can be replaced by the merged image of AF488, Cy3 and Cy5 channel.
- Prior to merging multiple image channels, it is crucial to address the variation in grayscale values that arise when different pseudocolors are directly converted to grayscale images. To ensure consistency and accurate representation of data, all channels should be converted to grayscale mode before the merging process. This standardization step is essential because different colors, when transformed to grayscale, can produce varying levels of brightness and contrast, potentially leading to misinterpretation of the relative intensities across channels.

## Step 2: Image registration



- The following Python code uses the `SimpleITK` library to call the `elastix` and `transformix` tools for rigid registration. Before running the code, please ensure that `elastix` and `transformix` are correctly installed, and the directories containing their executable files are added to the system’s environment variables so that the code can call them properly. In addition, you need to install the following modules in the notebook environment using the following commands: `pip install SimpleITK` ; `pip install SimpleITK-SimpleElastix`.
- The main function of the code is to align fluorescence images (DAPI, AF488, Cy3, and Cy5 channels) at different time points with a spatial reference (e.g., the image obtained at time point 1) to correct positional deviations between the images. It contains two main steps.
- `Step 1: registration of DAPI images`: rigidly register the DAPI images at different time points (T2_DAPI, T3_DAPI) with the reference image (T1_DAPI), and record the transformation parameters during the registration process.
- `Step 2: application of transformation parameters`: apply the transformation parameters for DAPI registration to other channel images (AF488, Cy3, Cy5) obtained at the same time point.

In [1]:
import SimpleITK as sitk
import os


def rigid_registration(fixed_image, moving_image):
    """
    Perform rigid registration between fixed and moving images.
    :param fixed_image: Fixed image for registration.
    :param moving_image: Moving image to be registered to the fixed image.
    :return: Transformed moving image and the transformation parameter map.
    """
    elastix_image_filter = sitk.ElastixImageFilter()
    elastix_image_filter.SetFixedImage(fixed_image)
    elastix_image_filter.SetMovingImage(moving_image)

    # Get the default rigid registration parameter map
    parameter_map = sitk.GetDefaultParameterMap("rigid")
    elastix_image_filter.SetParameterMap(parameter_map)

    # Execute the registration
    elastix_image_filter.Execute()

    # Get the result image and the transformation parameter map
    result_image = elastix_image_filter.GetResultImage()
    transform_map = elastix_image_filter.GetTransformParameterMap()

    return result_image, transform_map


def apply_transform(moving_image, transform_map):
    """
    Apply the transformation to the moving image using the given parameter map.
    :param moving_image: Moving image to be transformed.
    :param transform_map: Transformation parameter map.
    :return: Transformed moving image.
    """
    transformix_image_filter = sitk.TransformixImageFilter()
    transformix_image_filter.SetMovingImage(moving_image)
    transformix_image_filter.SetTransformParameterMap(transform_map)
    transformix_image_filter.Execute()
    return transformix_image_filter.GetResultImage()


def get_image_type(image_path):
    """
    Get the pixel type of the image
    :param image_path: Path to the image
    :return: Pixel type string
    """
    image = sitk.ReadImage(image_path)
    pixel_id = image.GetPixelID()
    
    if pixel_id == sitk.sitkUInt8:
        return sitk.sitkUInt8
    elif pixel_id == sitk.sitkUInt16:
        return sitk.sitkUInt16
    else:
        raise ValueError(f"Unsupported pixel type for image: {image_path}")


# Input image names
image_names = [
    "T1_AF488", "T1_Cy3", "T1_Cy5", "T1_merge",
    "T2_AF488", "T2_Cy3", "T2_Cy5", "T2_merge",
    "T3_AF488", "T3_Cy3", "T3_Cy5", "T3_merge"
]

# Directory where the input images are stored
input_dir = "D://FcTag decoding//1_input_image"
# Directory where the output images will be saved
output_dir = "D://FcTag decoding//2_image_registration"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Read the fixed image (T1_merge)
fixed_image_name = "T1_merge"
fixed_image_path = os.path.join(input_dir, f"{fixed_image_name}.tif")
fixed_image_type = get_image_type(fixed_image_path)
fixed_image = sitk.ReadImage(fixed_image_path, fixed_image_type)

# Dictionary to store transformation parameter maps
transformation_maps = {}

# First align T2_merge and T3_merge to T1_merge
for time_point in [2, 3]:
    merge_image_name = f"T{time_point}_merge"
    merge_image_path = os.path.join(input_dir, f"{merge_image_name}.tif")
    
    # Read image with original type
    image_type = get_image_type(merge_image_path)
    merge_image = sitk.ReadImage(merge_image_path, image_type)
    
    # Perform rigid registration
    aligned_merge_image, transform_map = rigid_registration(fixed_image, merge_image)
    
    # Convert back to original type
    aligned_merge_image = sitk.Cast(aligned_merge_image, image_type)
    
    # Save the aligned merge image
    aligned_output_path = os.path.join(output_dir, f"aligned_{merge_image_name}.tif")
    sitk.WriteImage(aligned_merge_image, aligned_output_path)
    
    # Store the transformation map for later use
    transformation_maps[time_point] = transform_map

# Apply transformations to AF488, Cy3, Cy5 channels using corresponding merge image transforms
for time_point in [2, 3]:
    # Get the transformation map from the corresponding merge image
    transform_map = transformation_maps[time_point]
    
    # Apply the transformation to AF488, Cy3, Cy5 channels
    for channel in ["AF488", "Cy3", "Cy5"]:
        channel_image_name = f"T{time_point}_{channel}"
        channel_image_path = os.path.join(input_dir, f"{channel_image_name}.tif")
        
        # Read image with original type
        image_type = get_image_type(channel_image_path)
        channel_image = sitk.ReadImage(channel_image_path, image_type)
        
        # Apply the transformation
        aligned_channel_image = apply_transform(channel_image, transform_map)
        
        # Convert back to original type
        aligned_channel_image = sitk.Cast(aligned_channel_image, image_type)
        
        # Save the aligned channel image
        aligned_output_path = os.path.join(output_dir, f"aligned_{channel_image_name}.tif")
        sitk.WriteImage(aligned_channel_image, aligned_output_path)

print("Registration and transformation completed.")

Registration and transformation completed.


## Step 3: Trim aligned images to the same size

- This step is accomplished with the help of Fiji using the following Java code：
- Fiji software can be downloaded from the [official Fiji software website](https://imagej.net/software/fiji/downloads).

In [ ]:
# To generate a customized code: Open Fiji → Plugins → Macro → Record
# To run the code: Open Fiji → Process → Batch → Macro
# Input folder: D://FcTag decoding//3_Fiji_trim_input
# Output folder: D://FcTag decoding//3_Fiji_trim_output
//setTool("rectangle");
makeRectangle(60, 32, 960, 960);
# To choose an appropriate size to crop: Open Fiji → open one of the image → choose Rectangle → the size will be shown in the software: x=4.18 (129), y=5.11 (315), w=0.50 (1024), h=0.46 (1024);
# The cropped dimensions will be: 1024px* 1024px, as illustrated by the values w=0.50 (1024), h=0.46 (1024);
run("Crop");

## Step 4: Extra color-changing information as one-dimensional array for each pixel

For each pixel in the FcTag image, the color-changing information over different time points is stored in a one-dimensional array_m and displayed as follows: 
```
array_m = np.array([T1_AF488, T1_Cy3, T1_Cy5, T2_AF488, T2_Cy3, T2_Cy5, T3_AF488, T3_Cy3, T3_Cy5])
```

- `T1_AF488`: AF488 channel obtained at time point 1 (0 min in this study)
- `T1_Cy3`: Cy3 channel obtained at time point 1 (0 min in this study)
- `T1_Cy5`: Cy5 channel obtained at time point 1 (0 min in this study)
- `T2_AF488`: AF488 channel obtained at time point 2 (15 min in this study)
- `T2_Cy3`: Cy3 channel obtained at time point 2 (15 min in this study)
- `T2_Cy5`: Cy5 channel obtained at time point 2 (15 min in this study)
- `T3_AF488`: AF488 channel obtained at time point 3 (30 min in this study)
- `T3_Cy3`: Cy3 channel obtained at time point 3 (30 min in this study)
- `T3_Cy5`: Cy5 channel obtained at time point 3 (30 min in this study)

The color-changing information for each pixel is then stored in a worksheet, where the cell coordinates correspond to the pixel coordinates in the image.

In [2]:
import os
import pandas as pd
from PIL import Image
from openpyxl import Workbook

# image file path
image_files = [
    'D://FcTag decoding//4_extra_color_information//T1_AF488.tif',
    'D://FcTag decoding//4_extra_color_information//T1_Cy3.tif',
    'D://FcTag decoding//4_extra_color_information//T1_Cy5.tif',
    'D://FcTag decoding//4_extra_color_information//T2_AF488.tif',
    'D://FcTag decoding//4_extra_color_information//T2_Cy3.tif',
    'D://FcTag decoding//4_extra_color_information//T2_Cy5.tif',
    'D://FcTag decoding//4_extra_color_information//T3_AF488.tif',
    'D://FcTag decoding//4_extra_color_information//T3_Cy3.tif',
    'D://FcTag decoding//4_extra_color_information//T3_Cy5.tif'
]

# create a new workbook
wb = Workbook()
ws = wb.active
ws.title = "Gray Values"

# create a dictionary to store the grayscale values of coordinates
gray_value_dict = {}

# iterate through each image file
for image_file in image_files:
    with Image.open(image_file) as img:
        
        # get image size
        width, height = img.size
        
        # extract the grayscale value of each pixel
        for y in range(height):
            for x in range(width):
                value = img.getpixel((x, y))
                
                # use the coordinates (x, y) as the key in the dictionary
                if (x, y) not in gray_value_dict:
                    gray_value_dict[(x, y)] = []
                gray_value_dict[(x, y)].append(value)

# write the grayscale values to the worksheet
for (x, y), values in gray_value_dict.items():
    # Combine grayscale values with the same coordinates into a one-dimensional array
    combined_values = ', '.join(map(str, values))
    ws.cell(row=y + 1, column=x + 1, value=combined_values)

# write the grayscale values to the worksheet
output_file = 'D://FcTag decoding//4_extra_color_information//color_changing_data.xlsx'
wb.save(output_file)

print(f"grayscale values have been exported to {output_file}")

grayscale values have been exported to D://FcTag decoding//4_extra_color_information//color_changing_data.xlsx


## Step 5. Linear unmixing

The fundamental concept of linear unmixing is based on the following formula:

 `array_m = C1*array_1 + C2*array_2 + …… + C27*array_27`

- `array_m`：Color-changing information of each pixel.
- `array_1` ~ `array_27`: Reference spectrum generated in section 5.
- `contribution factor (C)`: Contribution of array_x in making up array_m, which also corresponds to the relative fluorescence intensity of the recontrasted image.
- The match score, represented by the Mean Squared Error (MSE), can be fine-tuned to achieve optimal results. In our study, a threshold of MSE < 20 is applied, and values exceeding this threshold are deemed unreliable and excluded from analysis (i.e., set to zero). While these nullified pixels may slightly impact the visual quality of the decoded image, their effect on subsequent biological analyses is minimal and negligible.

### 5.1 Linear unmixing of retianl proteins

In [ ]:
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.metrics import mean_squared_error
from scipy.optimize import nnls
import openpyxl

# Read the color-changing information of each pixel
input_df = pd.read_excel('D://FcTag decoding//5_linear_unmixing//color_changing_data.xlsx', header=None)

# Spectrum-transforming fluorescent barcodes assigned to each protein
arrays = [
  np.array([0, 0, 1, 0, 0, 1, 1, 0, 0]), #1_β-Tubulin
  np.array([0, 1, 0, 0, 1, 0, 1, 0, 0]), #2_Vimentin
  np.array([1, 0, 0, 1, 0, 0, 0, 1, 0]), #3_PKC-α
  np.array([0, 0, 1, 0, 0, 1, 0, 0, 1]), #4_Chx10
  np.array([1, 0, 0, 0, 0, 1, 0, 1, 0]), #5_SV2
  np.array([0, 0, 1, 0, 1, 0, 0, 1, 0]), #6_Synapsin
]

def find_combinations(array_m, arrays, max_arrays=6):
    best_combination = None
    best_constants = None
    best_score = float('inf')

    for n in range(1, max_arrays + 1):
        for combo in combinations(enumerate(arrays), n):
            indices, selected_arrays = zip(*combo)
            
            A = np.array(selected_arrays).T
            b = array_m
            try:
                constants, _ = nnls(A, b)
                predicted = np.dot(A, constants)
                score = mean_squared_error(array_m, predicted)
                
                if score < best_score:
                    best_score = score
                    best_combination = indices
                    best_constants = constants
            except ValueError:
                continue

    return best_combination, best_constants, best_score

# Creat a new Excel file to store results
workbook = openpyxl.Workbook()

# Define the sheet with array and protein name
sheet_names = ['1_Tubulin', '2_Vimentin', '3_PKC', '4_Chx10', '5_SV2', '6_Synapsin', 'Match Score']

# Create an empty DataFrame for each sheet and fill it with 0.0
results = {name: pd.DataFrame(0.0, index=input_df.index, columns=input_df.columns, dtype=float) for name in sheet_names}

# Calculate the optimal combination for each cell
for row in range(input_df.shape[0]):
    for col in range(input_df.shape[1]):
        cell_value = input_df.iloc[row, col]
        if pd.notna(cell_value):  # Check if the cell is empty
            array_m = np.array([float(x) for x in str(cell_value).split(',')])  # Convert the cell value to an array
            if len(array_m) == len(arrays[0]):  # Ensure length matches
                combination, constants, score = find_combinations(array_m, arrays)
                
                if score <= 20:
                    for i, const in zip(combination, constants):
                        results[sheet_names[i]].iloc[row, col] = const
                
                results['Match Score'].iloc[row, col] = score
            else:
                print(f"Warning: Mismatch in array length at row {row+1}, column {col+1}")

# Write the results to Excel file
with pd.ExcelWriter('D://FcTag decoding//5_linear_unmixing//contribution_factor.xlsx', engine='openpyxl') as writer:
    for sheet_name, df in results.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

print("Results have been written to contribution_factor.xlsx")

### 5.2 Linear unmixing of breast cancer proteins

In [4]:
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.metrics import mean_squared_error
from scipy.optimize import nnls
import openpyxl

# Read the Excel file (raw_data.xlsx)
input_df = pd.read_excel('D://FcTag decoding//5_linear_unmixing//color_changing_data.xlsx', header=None)

# color-changing barcode
arrays = [
  np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]), #01_CD3
  np.array([1, 0, 0, 0, 0, 1, 0, 1, 0]), #02_CD4
  np.array([0, 1, 0, 0, 0, 1, 1, 0, 0]), #03_CD8a
  np.array([0, 0, 1, 0, 0, 1, 0, 1, 0]), #04_CD19
  np.array([0, 1, 0, 1, 0, 0, 0, 0, 1]), #05_CD20
  np.array([0, 1, 0, 0, 1, 0, 0, 0, 1]), #06_CD45RA
  np.array([1, 0, 0, 0, 1, 0, 1, 0, 0]), #07_CD45RO
  np.array([0, 1, 0, 0, 0, 1, 0, 0, 1]), #08_CD56
  np.array([1, 0, 0, 0, 0, 1, 1, 0, 0]), #09_CD68
  np.array([0, 0, 1, 1, 0, 0, 1, 0, 0]), #10_CD80
  np.array([0, 0, 1, 0, 1, 0, 0, 1, 0]), #11_CD163
  np.array([0, 1, 0, 0, 1, 0, 0, 1, 0]), #12_Foxp3
  np.array([1, 0, 0, 1, 0, 0, 0, 1, 0]), #13_GZMB
  np.array([0, 1, 0, 1, 0, 0, 1, 0, 0]), #14_MPO
  np.array([0, 1, 0, 0, 0, 1, 0, 1, 0]), #15_PIP
  np.array([0, 0, 1, 1, 0, 0, 0, 0, 1]), #16_HER2
  np.array([1, 0, 0, 0, 0, 1, 0, 0, 1]), #17_CK56
  np.array([1, 0, 0, 1, 0, 0, 0, 0, 1]), #18_CK19
  np.array([0, 0, 1, 0, 1, 0, 1, 0, 0]), #19_EGFR
  np.array([0, 0, 1, 0, 0, 1, 1, 0, 0]), #20_ER
  np.array([0, 0, 1, 1, 0, 0, 0, 1, 0]), #21_PR
  np.array([0, 0, 1, 0, 0, 1, 0, 0, 1]), #22_AR
  np.array([1, 0, 0, 1, 0, 0, 1, 0, 0]), #23_Ki67
  np.array([0, 1, 0, 0, 1, 0, 0, 1, 0]), #24_p63
  np.array([0, 0, 1, 0, 1, 0, 0, 0, 1]), #25_CD31
  np.array([1, 0, 0, 0, 1, 0, 0, 1, 0]), #26_SMA
  np.array([0, 1, 0, 0, 1, 0, 1, 0, 0]), #27_PDPN
]

def find_combinations(array_m, arrays, max_arrays=4):
    best_combination = None
    best_constants = None
    best_score = float('inf')

    for n in range(1, max_arrays + 1):
        for combo in combinations(enumerate(arrays), n):
            indices, selected_arrays = zip(*combo)
            
            A = np.array(selected_arrays).T
            b = array_m
            try:
                constants, _ = nnls(A, b)
                predicted = np.dot(A, constants)
                score = mean_squared_error(array_m, predicted)
                
                if score < best_score:
                    best_score = score
                    best_combination = indices
                    best_constants = constants
            except ValueError:
                continue

    return best_combination, best_constants, best_score

# Creat a new Excel file to store results
workbook = openpyxl.Workbook()

# Define the sheet with array and protein name
sheet_names = ['01_CD3', '02_CD4', '03_CD8a', '04_CD19', '05_EGFR', '06_CD45RA', '07_CD45RO', '08_CD56', '09_CD68', 
               '10_CD80', '11_CD163', '12_Foxp3', '13_GZMB', '14_MPO', '15_PIP', '16_HER2', '17_CK56', '18_CK19', 
               '19_EGFR', '20_ER', '21_PR', '22_AR', '23_Ki67', '24_p63', '25_CD31', '26_SMA', '27_PDPN', 'Match Score']

# Create an empty DataFrame for each sheet and fill it with 0.0
results = {name: pd.DataFrame(0.0, index=input_df.index, columns=input_df.columns, dtype=float) for name in sheet_names}

# Calculate the optimal combination for each cell
for row in range(input_df.shape[0]):
    for col in range(input_df.shape[1]):
        cell_value = input_df.iloc[row, col]
        if pd.notna(cell_value):  # Check if the cell is empty
            array_m = np.array([float(x) for x in str(cell_value).split(',')])  # Convert the cell value to an array
            if len(array_m) == len(arrays[0]):  # Ensure length matches
                combination, constants, score = find_combinations(array_m, arrays)
                
                if score <= 20:
                    for i, const in zip(combination, constants):
                        results[sheet_names[i]].iloc[row, col] = const
                
                results['Match Score'].iloc[row, col] = score
            else:
                print(f"Warning: Mismatch in array length at row {row+1}, column {col+1}")

# Write the results to Excel file
with pd.ExcelWriter('D://FcTag decoding//5_linear_unmixing//contribution_factor.xlsx', engine='openpyxl') as writer:
    for sheet_name, df in results.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

print("Results have been written to output_results.xlsx")

Results have been written to output_results.xlsx


### 5.3 Linear unmixing based on biological prior knowledge

In this section, we introduce prioritized marker combinations based on established knowledge of staining patterns. For instance, we prioritize the combination of array_1 (CD3), array_2 (CD4), and array_24 (CD45RA) for identifying naive CD4+ T cells. Our algorithm initially searches for optimal matches within these predefined, high-priority combinations. If no satisfactory matches are found (defined as combinations with a Mean Squared Error below 10), the algorithm expands its search beyond these initial constraints. Importantly, the Mean Squared Error threshold is adaptable and can be fine-tuned based on empirical results.

In [19]:
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.metrics import mean_squared_error
from scipy.optimize import nnls
import openpyxl

# Read the Excel file (raw_data.xlsx)
input_df = pd.read_excel('D://Linear unmixing//6_linear unmixing//raw_data.xlsx', header=None)

# Reference spectrum (generated in section 5)
arrays = [
    np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]), #01_CD3
    np.array([1, 0, 0, 0, 0, 1, 0, 1, 0]), #02_CD4
    np.array([0, 1, 0, 0, 0, 1, 1, 0, 0]), #03_CD8a
    np.array([0, 0, 1, 0, 0, 1, 0, 1, 0]), #04_CD19
    np.array([0, 1, 0, 1, 0, 0, 0, 0, 1]), #05_CD20
    np.array([0, 1, 0, 0, 1, 0, 0, 0, 1]), #06_CD45RA
    np.array([1, 0, 0, 0, 1, 0, 1, 0, 0]), #07_CD45RO
    np.array([0, 1, 0, 0, 0, 1, 0, 0, 1]), #08_CD56
    np.array([1, 0, 0, 0, 0, 1, 1, 0, 0]), #09_CD68
    np.array([0, 0, 1, 1, 0, 0, 1, 0, 0]), #10_CD80
    np.array([0, 0, 1, 0, 1, 0, 0, 1, 0]), #11_CD163
    np.array([0, 1, 0, 0, 1, 0, 0, 1, 0]), #12_Foxp3
    np.array([1, 0, 0, 1, 0, 0, 0, 1, 0]), #13_GZMB
    np.array([0, 1, 0, 1, 0, 0, 1, 0, 0]), #14_MPO
    np.array([0, 1, 0, 0, 0, 1, 0, 1, 0]), #15_PIP
    np.array([0, 0, 1, 1, 0, 0, 0, 0, 1]), #16_HER2
    np.array([1, 0, 0, 0, 0, 1, 0, 0, 1]), #17_CK56
    np.array([1, 0, 0, 1, 0, 0, 0, 0, 1]), #18_CK19
    np.array([0, 0, 1, 0, 1, 0, 1, 0, 0]), #19_EGFR
    np.array([0, 0, 1, 0, 0, 1, 1, 0, 0]), #20_ER
    np.array([0, 0, 1, 1, 0, 0, 0, 1, 0]), #21_PR
    np.array([0, 0, 1, 0, 0, 1, 0, 0, 1]), #22_AR
    np.array([1, 0, 0, 1, 0, 0, 1, 0, 0]), #23_Ki67
    np.array([0, 1, 0, 0, 1, 0, 0, 1, 0]), #24_p63
    np.array([0, 0, 1, 0, 1, 0, 0, 0, 1]), #25_CD31
    np.array([1, 0, 0, 0, 1, 0, 0, 1, 0]), #26_SMA
    np.array([0, 1, 0, 0, 1, 0, 1, 0, 0]), #27_PDPN
]

# Prioritized protein combinations. The numbers enclosed in parentheses signify a potential combination. These numbers correspond to the array index minus one (e.g., 0 denotes array_1).
specified_combinations = [
    [(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,), (13,), (14,), (15,), (16,), (17,), (18,), (19,), (20,), (21,), (22,), (23,), (24,), (25,), (26,)],
    [(0, 1), (0, 23), (1, 23), (0, 1, 23), (0, 1), (0, 6), (1, 6), (0, 1, 6)],
    [(8, 24)],
    [(0, 2), (0, 23), (2, 23), (0, 2, 23)],
    [(0, 2), (0, 6), (2, 6), (0, 2, 6)],
    [(0, 2), (0, 18), (0, 23), (2, 18), (2, 23), (18, 23), (0, 2, 18), (0, 2, 23), (0, 18, 23), (2, 18, 23), (0, 2, 18, 23)],
    [(0, 2), (0, 18), (0, 6), (2, 18), (2, 6), (18, 6), (0, 2, 18), (0, 2, 6), (0, 18, 6), (2, 18, 6), (0, 2, 18, 6)],
    [(21, 3)],
    [(7, 16), (7, 17), (7, 16, 17)],
    [(24, 25)],
    [(22, 5), (22, 26), (22, 24), (5, 26), (5, 24), (26, 24), (22, 5, 26), (22, 5, 24), (22, 26, 24), (5, 26, 24), (22, 5, 26, 24)],
    [(9, 13), (9, 14), (9, 10), (9, 4), (13, 14), (13, 10), (13, 4), (14, 10), (14, 4), (10, 4)],
    [(9, 13, 14), (9, 13, 10), (9, 13, 4), (9, 14, 10), (9, 14, 4), (9, 10, 4), (13, 14, 10), (13, 14, 4), (13, 10, 4), (14, 10, 4)],
    [(9, 13, 14, 10), (9, 13, 14, 4), (9, 13, 10, 4), (9, 14, 10, 4), (13, 14, 10, 4)],
    [(9, 13, 14, 10, 4)],
    [(11, 12)]
]

def find_combinations(array_m, arrays, max_arrays=5):
    best_combination = None
    best_constants = None
    best_score = float('inf')

    # Searches for optimal matches within these predefined, high-priority combinations
    specified_best_score = float('inf')
    specified_best_combination = None
    specified_best_constants = None
    for combos in specified_combinations:
        for combo in combos:
            indices = combo
            selected_arrays = [arrays[i] for i in indices]
            A = np.array(selected_arrays).T
            try:
                constants, _ = nnls(A, array_m)
                predicted = np.dot(A, constants)
                score = mean_squared_error(array_m, predicted)
                
                if score <= 10 and score < specified_best_score:
                    specified_best_score = score
                    specified_best_combination = indices
                    specified_best_constants = constants
            except ValueError:
                continue

    if specified_best_combination is not None:
        best_combination = specified_best_combination
        best_constants = specified_best_constants
        best_score = specified_best_score
    else:
        # If no satisfactory matches are found, the algorithm expands its search beyond these initial constraints
        for n in range(1, max_arrays + 1):
            for combo in combinations(enumerate(arrays), n):
                indices, selected_arrays = zip(*combo)
                A = np.array(selected_arrays).T
                try:
                    constants, _ = nnls(A, array_m)
                    predicted = np.dot(A, constants)
                    score = mean_squared_error(array_m, predicted)
                    
                    if score < best_score:
                        best_score = score
                        best_combination = indices
                        best_constants = constants
                except ValueError:
                    continue

    return best_combination, best_constants, best_score

# Creat a new Excel file to store results
workbook = openpyxl.Workbook()

# Define the sheet with array and protein name
sheet_names = ['01_CD3', '02_CD4', '03_CD8a', '04_CD19', '05_CD20', '06_CD45RA', '07_CD45RO', '08_CD56', '09_CD68', 
               '10_CD80', '11_CD163', '12_Foxp3', '13_GZMB', '14_MPO', '15_PIP', '16_HER2', '17_CK56', '18_CK19', 
               '19_EGFR', '20_ER', '21_PR', '22_AR', '23_Ki67', '24_p63', '25_CD31', '26_SMA', '27_PDPN', 'Match Score']

# Create an empty DataFrame for each sheet and fill it with 0.0
results = {name: pd.DataFrame(0.0, index=input_df.index, columns=input_df.columns, dtype=float) for name in sheet_names}

# Use NumPy's vectorized operations to process input_df.
input_array = input_df.to_numpy()
for row in range(input_df.shape[0]):
    for col in range(input_df.shape[1]):
        cell_value = input_array[row, col]
        if pd.notna(cell_value):  # Check if the cell is empty
            array_m = np.array([float(x) for x in str(cell_value).split(',')])  # Convert the cell value to an array
            if len(array_m) == len(arrays[0]):  # Ensure length matches
                combination, constants, score = find_combinations(array_m, arrays)
                
                if score <= 100:
                    for i, const in zip(combination, constants):
                        results[sheet_names[i]].iloc[row, col] = const
                
                results['Match Score'].iloc[row, col] = score
            else:
                print(f"Warning: Mismatch in array length at row {row+1}, column {col+1}")

# Write the results to Excel file
with pd.ExcelWriter('D://Linear unmixing//6_linear unmixing//contribution_factor.xlsx', engine='openpyxl') as writer:
    for sheet_name, df in results.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

## Step 6: Image subtraction

Another approach to decode proteins from raw FcTag images is ‘image subtraction’. This method relies on maintaining consistent fluorescent intensities across different channels while avoiding overexposure, which is prerequisite for accurate target decoding. Based on this, when two images are merged, the fluorescence intensity of the combined image equals the sum of the intensities of the individual images (Image1+2 = Image1 + Image2). Conversely, by subtracting one image (Image1) from the merged image (Image1+2), the other image (Image2) can be resolved. These images can be any mathematically subtractable images, and the resolved image can also serve as a new base image for subsequent subtraction process. 

In [13]:
import cv2
import numpy as np
import os
import shutil

def subtract_images(img_path1, img_path2, output_path):
    """Read two images, perform subtraction (only keep parts where img1 > img2), and save the result"""
    # Read images
    ImageC1 = cv2.imread(img_path1, cv2.IMREAD_GRAYSCALE)
    ImageC2 = cv2.imread(img_path2, cv2.IMREAD_GRAYSCALE)
    
    if ImageC1 is None or ImageC2 is None:
        print(f"Error: Failed to read images. {img_path1} or {img_path2} does not exist")
        return False
    
    # Ensure both images have the same dimensions
    if ImageC1.shape != ImageC2.shape:
        print(f"Error: Image dimensions do not match. {img_path1}: {ImageC1.shape}, {img_path2}: {ImageC2.shape}")
        return False
    
    # Get image dimensions
    height, width = ImageC1.shape
    
    # Create an empty matrix to store new grayscale values
    new_gray_values = np.zeros((height, width), dtype=np.uint8)
    
    # Traverse each pixel
    for i in range(height):
        for j in range(width):
            if ImageC1[i, j] > ImageC2[i, j]:
                new_gray_values[i, j] = ImageC1[i, j] - ImageC2[i, j]
            else:
                new_gray_values[i, j] = 0
    
    # Save the result image
    cv2.imwrite(output_path, new_gray_values)
    print(f"Successfully saved image to: {output_path}")
    return True

def copy_and_rename_tif_file(source_file, target_file):
    """Copy and rename a TIF file while preserving metadata"""
    try:
        # Check if the source file exists
        if not os.path.exists(source_file):
            print(f"Error: Source file {source_file} does not exist")
            return False
        
        # Check if the target file already exists
        if os.path.exists(target_file):
            print(f"Error: Target file {target_file} already exists")
            return False
        
        # Create the directory for the target file if it doesn't exist
        target_dir = os.path.dirname(target_file)
        os.makedirs(target_dir, exist_ok=True)
        
        # Copy the file (preserves metadata)
        shutil.copy2(source_file, target_file)
        print(f"Successfully copied file: {source_file} -> {target_file}")
        return True
        
    except Exception as e:
        print(f"Error: An exception occurred during file copying: {e}")
        return False

# Set the base directory
base_dir = "D://FcTag decoding//6_Image_subtraction"

# Perform image subtraction operations
subtract_images(
    os.path.join(base_dir, "T1_AF488.tif"),
    os.path.join(base_dir, "T2_AF488.tif"),
    os.path.join(base_dir, "01_SV2.tif")
)

subtract_images(
    os.path.join(base_dir, "T2_Cy3.tif"),
    os.path.join(base_dir, "T1_Cy3.tif"),
    os.path.join(base_dir, "02_Synapse.tif")
)

subtract_images(
    os.path.join(base_dir, "T3_AF488.tif"),
    os.path.join(base_dir, "T1_Cy3.tif"),
    os.path.join(base_dir, "03_Tubulin.tif")
)

# Perform file copying operations
copy_and_rename_tif_file(
    os.path.join(base_dir, "T1_Cy3.tif"),
    os.path.join(base_dir, "04_Vimentin.tif")
)

copy_and_rename_tif_file(
    os.path.join(base_dir, "T2_AF488.tif"),
    os.path.join(base_dir, "05_PKC-α.tif")
)

copy_and_rename_tif_file(
    os.path.join(base_dir, "T3_Cy5.tif"),
    os.path.join(base_dir, "06_Chx10.tif")
)

Successfully saved image to: D://FcTag decoding//6_Image_subtraction\01_SV2.tif
Successfully saved image to: D://FcTag decoding//6_Image_subtraction\02_Synapse.tif
Successfully saved image to: D://FcTag decoding//6_Image_subtraction\03_Tubulin.tif
Successfully copied file: D://FcTag decoding//6_Image_subtraction\T1_Cy3.tif -> D://FcTag decoding//6_Image_subtraction\04_Vimentin.tif
Successfully copied file: D://FcTag decoding//6_Image_subtraction\T2_AF488.tif -> D://FcTag decoding//6_Image_subtraction\05_PKC-α.tif
Successfully copied file: D://FcTag decoding//6_Image_subtraction\T3_Cy5.tif -> D://FcTag decoding//6_Image_subtraction\06_Chx10.tif


True

## Step 7: Image display

- To assign the decoded image (grayscale) with `pseudo color`: Open zen 3.11 software → Dimensions → Channels.
- To `merge images` of different proteins: Open zen 3.11 software → Processing → Method → Add channes.
- zen 3.11 software can be downloaded from the [official Zeiss software website](https://www.zeiss.com/microscopy/zh/products/software/zeiss-zen.html).
- The contribution factor decoded by linear unmixing can be displayed as a grayscale image (.tif) using the following code.

- for 8-bit image display

In [5]:
import openpyxl
from PIL import Image
import numpy as np

def excel_to_tiff(excel_path, output_dir):
    # Open the Excel file
    workbook = openpyxl.load_workbook(excel_path)

    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]

        # Get the dimensions of the Excel sheet
        max_row = sheet.max_row
        max_col = sheet.max_column

        # Create a numpy array to store grayscale values, with data type np.uint8 (8-bit)
        image_array = np.zeros((max_row, max_col), dtype=np.uint8)

        # Read values from the Excel sheet and populate the array
        for row in range(1, max_row + 1):
            for col in range(1, max_col + 1):
                cell_value = sheet.cell(row=row, column=col).value
                if cell_value is not None:
                    # Ensure the value is within the range of an 8-bit unsigned integer (0-255)
                    cell_value = max(0, min(int(cell_value), 255))
                    image_array[row - 1, col - 1] = cell_value

        # Create an image with 8-bit mode ('L' for grayscale)
        image = Image.fromarray(image_array, mode='L')

        # Construct the output file path, naming it after the sheet name
        tiff_path = f"{output_dir}/{sheet_name}.tif"

        # Save the image as a TIFF file
        image.save(tiff_path, format='TIFF')
        print(f"TIFF image has been saved to {tiff_path}")

# Specify the input Excel file path and output directory path
excel_file_path = r"D://FcTag decoding//7_image_display//contribution_factor.xlsx"  # Replace with your Excel file path
output_directory = r"D://FcTag decoding//7_image_display"  # Replace with the directory path where you want to save the TIFF files

# Call the function
excel_to_tiff(excel_file_path, output_directory)

TIFF image has been saved to D://FcTag decoding//7_image_display/01_CD3.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/02_CD4.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/03_CD8a.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/04_CD19.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/05_EGFR.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/06_CD45RA.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/07_CD45RO.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/08_CD56.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/09_CD68.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/10_CD80.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/11_CD163.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/12_Foxp3.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/13_GZMB

- for 16-bit image display

In [6]:
import openpyxl
from PIL import Image
import numpy as np

def excel_to_tiff(excel_path, output_dir):
    # Open the Excel file
    workbook = openpyxl.load_workbook(excel_path)

    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]

        # Get the dimensions of the Excel sheet
        max_row = sheet.max_row
        max_col = sheet.max_column

        # Create a numpy array to store grayscale values, with data type np.uint16
        image_array = np.zeros((max_row, max_col), dtype=np.uint16)

        # Read values from the Excel sheet and populate the array
        for row in range(1, max_row + 1):
            for col in range(1, max_col + 1):
                cell_value = sheet.cell(row=row, column=col).value
                if cell_value is not None:
                    # Ensure the value is within the range of a 16-bit unsigned integer
                    cell_value = max(0, min(int(cell_value), 65535))
                    image_array[row - 1, col - 1] = cell_value

        # Create an image
        image = Image.fromarray(image_array, mode='I;16')

        # Construct the output file path, naming it after the sheet name
        tiff_path = f"{output_dir}/{sheet_name}.tif"

        # Save the image as a TIFF file
        image.save(tiff_path, format='TIFF')
        print(f"TIFF image has been saved to {tiff_path}")

# Specify the input Excel file path and output directory path
excel_file_path = r"D://FcTag decoding//7_image_display//contribution_factor.xlsx"  # Replace with your Excel file path
output_directory = r"D://FcTag decoding//7_image_display"  # Replace with the directory path where you want to save the TIFF files

# Call the function
excel_to_tiff(excel_file_path, output_directory)

TIFF image has been saved to D://FcTag decoding//7_image_display/01_CD3.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/02_CD4.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/03_CD8a.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/04_CD19.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/05_EGFR.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/06_CD45RA.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/07_CD45RO.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/08_CD56.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/09_CD68.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/10_CD80.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/11_CD163.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/12_Foxp3.tif
TIFF image has been saved to D://FcTag decoding//7_image_display/13_GZMB